In [1]:
from pyspark.sql import SparkSession

#spark = SparkSession.builder.appName("WDC-DataFrame").getOrCreate()
#créer une session dans le master
spark = SparkSession.builder \
    .master("spark://172.20.53.96:7077") \
    .appName("WDC-Pascal-test") \
    .config("spark.executor.memory","20g") \
    .config("spark.driver.memory","20g") \
    .getOrCreate()
#spark = SparkSession.builder.master("local").appName("WDC-Pascaltest").getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions",1000)


#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = 'bbd95ea3c1174caa88345404b84e458f'
aws_secret_access_key = 'eaf2a72ecf9845f583af7f3513c44f25'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/26 05:04:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/26 05:04:26 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count
from pyspark.sql import functions as F

from pyspark.sql import Row
import re
from urllib.parse import urlparse


line='_:nb53a24408607424384c1357880ce1bc7xb1 <http://schema.org/value> "8,840 ft / 2,694 m" <https://peakery.com/dragontail-peak-washington/>   .'
quad_motif=re.compile(r'(.+)\s(<.+>)\s(.+)<(.+)>\s+.')

def parseQ(l):
  result=quad_motif.match(l)
  #print(result.groups())
  pred=result.group(2)
  if result.group(2)=="<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>":
      pred="isa:"+result.group(3)
  domain=urlparse(result.group(4)).netloc
  return Row(subject=result.group(1),predicate=pred,prov=domain,hashdom=hash(domain)%10)

print(parseQ(line))

#lines = spark.sparkContext.textFile("s3a://test/")
lines = spark.sparkContext.textFile("s3a://wdc/")

sp=lines.map(lambda l: parseQ(l)).toDF()
sp.show(truncate=0)

sp = sp.withColumn("predicate", F.regexp_replace(F.col("predicate"), "([Hh][Tt][Tt][Pp][Ss]?://)?([Ww]{3}\.)?", ""))
sp.show(truncate=0)

sp.createOrReplaceTempView("Super")
#spark.sql("select count(*) as count from Super").show()

Row(subject='_:nb53a24408607424384c1357880ce1bc7xb1', predicate='<http://schema.org/value>', prov='peakery.com', hashdom=8)
23/03/26 05:04:29 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+----------------------------------------+-------------------------------------+-----------------+-------+
|subject                                 |predicate                            |prov             |hashdom|
+----------------------------------------+-------------------------------------+-----------------+-------+
|<http://1000lifehacks.com/#organization>|isa:<http://schema.org/Organization> |1000lifehacks.com|1      |
|<http://1000lifehacks.com/#organization>|<http://schema.org/image>            |1000lifehacks.com|1      |
|<http://1000lifehacks.com/#organization>|<http://schema.org/logo>             |1000lifehacks.com|1      |
|<http://1000lifehacks.com/#organization>|<http://schema.org/name>             |1000lifehacks.com|1      |
|<http://1000lifehacks.com/#organization>|<http://schema.org/sameAs>           |1000lifehacks.com|1      |
|<http://1000lifehacks.com/#organization>|<http://schema.org/sameAs>           |1000lifehacks.com|1      |
|<http://1000lifehacks.com/#organizat

In [ ]:
import pyspark.sql.functions as f

for i in range(10):
    hashdom_val = i
    file_name = f"hashdom{hashdom_val+1}-10"
    cset2 = spark.sql(f"select subject, concat_ws('|',sort_array(collect_set(predicate))) as pset FROM Super where hashdom={hashdom_val} group by  subject ").cache()
    cset2.show(truncate=200)
    print(cset2.count())

    result2 = cset2.groupby("pset").agg(f.count(cset2.subject).alias('count')).sort(f.desc("count"))
    result2.show(truncate=0)

    result2.write.option("header",True) \
        .mode("overwrite") \
        .csv(f"s3a://test-out/wdc-httpswww/{file_name}")
    
    # clear variables from memory
    cset2.unpersist()
    result2.unpersist()
    del cset2, result2